In [1]:
import pyomo.environ as pyo

# Define sets
I = ['Wheat', 'Corn', 'Sugar beet']

# Define parameters
A = 500
D = {'Wheat': 200, 'Corn': 240, 'Sugar beet': 0} # Note: We assume no demand for sugar beet itself, only selling 
SP = {'Wheat': 170, 'Corn': 150, 'Sugar beet': 36, 'Sugar beet excess': 10}
PP = {'Wheat': 238, 'Corn': 210, 'Sugar beet': 0} # Note: Assuming no purchasing of sugar beet
Q = {'Sugar beet': 6000}
Y = {'Wheat': 2.5, 'Corn': 3, 'Sugar beet': 20}
C = {'Wheat': 150, 'Corn': 230, 'Sugar beet': 260}

# Create model
model = pyo.ConcreteModel()

# Define variables
model.X = pyo.Var(I, domain=pyo.NonNegativeReals) # Acres of land devoted to each crop
model.P = pyo.Var(I, domain=pyo.NonNegativeReals, initialize=0) # Tons of each crop purchased
model.S = pyo.Var(I, domain=pyo.NonNegativeReals) # Tons of each crop sold within quota
model.S_excess = pyo.Var(domain=pyo.NonNegativeReals) # Tons of sugar beet sold exceeding quota

# Define objective function
def objective_rule(model):
    return sum(C[i] * model.X[i] + PP[i] * model.P[i] - SP[i] * model.S[i] for i in I) - SP['Sugar beet excess'] * model.S_excess
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Define constraints
model.land_availability = pyo.Constraint(expr=sum(model.X[i] for i in I) <= A)

model.demand_wheat = pyo.Constraint(expr=Y['Wheat'] * model.X['Wheat'] + model.P['Wheat'] >= D['Wheat'])
model.demand_corn = pyo.Constraint(expr=Y['Corn'] * model.X['Corn'] + model.P['Corn'] >= D['Corn'])

model.sales_wheat = pyo.Constraint(expr=model.S['Wheat'] <= Y['Wheat'] * model.X['Wheat'] + model.P['Wheat'] - D['Wheat'])
model.sales_corn = pyo.Constraint(expr=model.S['Corn'] <= Y['Corn'] * model.X['Corn'] + model.P['Corn'] - D['Corn'])
model.sales_sugar_beet = pyo.Constraint(expr=model.S['Sugar beet'] <= Y['Sugar beet'] * model.X['Sugar beet'])

model.quota_sugar_beet = pyo.Constraint(expr=model.S['Sugar beet'] <= Q['Sugar beet'])

model.excess_sugar_beet = pyo.Constraint(expr=model.S_excess == Y['Sugar beet'] * model.X['Sugar beet'] - model.S['Sugar beet'])

# Solve the model
solver = pyo.SolverFactory('glpk') # You can choose a different solver if you prefer
solver.solve(model)

# Print results
print("Optimal solution:")
print(f"Objective value: ${model.objective():.2f}")
for i in I:
    print(f"Acres of {i}: {model.X[i]():.2f}")
    print(f"Tons of {i} purchased: {model.P[i]():.2f}")
    print(f"Tons of {i} sold: {model.S[i]():.2f}")
print(f"Tons of sugar beet sold exceeding quota: {model.S_excess():.2f}")

Optimal solution:
Objective value: $-118600.00
Acres of Wheat: 120.00
Tons of Wheat purchased: 0.00
Tons of Wheat sold: 100.00
Acres of Corn: 80.00
Tons of Corn purchased: 0.00
Tons of Corn sold: 0.00
Acres of Sugar beet: 300.00
Tons of Sugar beet purchased: 0.00
Tons of Sugar beet sold: 6000.00
Tons of sugar beet sold exceeding quota: 0.00
